# Encontrar E-mail

#### Imports

In [289]:
# Imports

from imap_tools import MailBox, AND
from pandas import DataFrame as df
import pandas as pd
from io import BytesIO
from datetime import datetime

#### Aceso Email

In [344]:
# Loga no Email

# https://www.systoolsgroup.com/imap/
# imap-mail.outlook.com
# usuario = "mateusramos.oficial@gmail.com"
# senha = "cejs pbde cfqn uwnt"

usuario = input("Digite seu email: ")
senha = input("Digite sua senha: ")

# meu_email = MailBox("imap.gmail.com").login(usuario, senha)
meu_email = MailBox("imap-mail.outlook.com").login(usuario, senha)


#### Encontra Destinatário Específico

* Coloca E-mail's em uma lista<br>
* Titulo Email - Data - Tem Anexo - Conteudo Anexo


In [345]:
# Filtra email com destinatário específico e com assunto específico.
# Armazena a quantidade de e-mails retornados
# quantidade_emails = print(len(list((lista_emails))))

destinatario = input("Digite o Email do Destinatáio: ")
assunto = input("Digite o assunto para filtrar: ")

# lista_emails = meu_email.fetch(AND(from_='brunacamsilva@gmail.com', subject='Python'))
lista_emails = meu_email.fetch(AND(from_=destinatario, subject=assunto))

lista_cabecalho = []

for email in lista_emails:
    # print(f"{email.subject} - {email.date}")
    titulo_email = f"{email.subject}"
    data = f"{email.date}"

    if email.attachments:
        for anexo in email.attachments:
            # print( f"{anexo.filename}" )
            anexo_titulo = f"{anexo.filename}"
            anexo_conteudo = anexo.payload
            
            lista_cabecalho.append((titulo_email, data, anexo_titulo, anexo_conteudo))
    
    else:
        lista_cabecalho.append((titulo_email, data, "Sem anexo", "Sem Conteudo"))

# print(" ")

# for titulo, data, anexo_titulo, anexo_conteudo in lista_cabecalho:
    # print(f"{titulo} - {data} - {anexo_titulo}")



#### Transforma vetor em Pandas

In [369]:
title = ['Titulo','Data','Anexo','Conteúdo']

resultado = df(lista_cabecalho, columns=title, index=None)
resultado = resultado.astype({'Titulo': 'object', 'Data': 'datetime64', 'Anexo': 'string'})
resultado = resultado.sort_values(by='Data',ascending=False)
resultado['Data'] = resultado['Data'].dt.strftime('%d-%m-%Y')

resultado.iloc[:, :-1]

,Titulo,Data,Anexo
3,Teste Python 3,17-08-2024,1_worksheet.xlsx
4,Teste Python 3,17-08-2024,2_worksheet.xlsx
2,Teste Python 2,17-08-2024,2_worksheet.xlsx
1,Teste Python 1,17-08-2024,1_worksheet.xlsx
0,Teste Python,17-08-2024,Sem anexo


#### Usuário escolhe o arquivo 

In [370]:
email_escolhido = int(input("Digite o código do email escolhido: "))

if 0 <= email_escolhido < len(lista_cabecalho):
    print(email_escolhido)
    anexo_escolhido = lista_cabecalho[email_escolhido][3]

3


#### Transforma conteúdo byte do Anexo em Pandas

In [371]:
dados_em_bytes = anexo_escolhido
buffer = BytesIO(dados_em_bytes)
planilha_anexo = pd.read_excel(buffer)

,ID,NOME,QUANTIDADE,ASDAD,ASDASD,ASDASD.1,PERIODO,ASDSAD
0,1,CAIO,5,asdsd,asdsad,asdsad,JUNHO,asdsad
1,2,BUSSCAR,3,asdasd,asdsad,asdasd,JUNHO,asdasd
2,3,FIBERBUS,5,asdsad,asdsad,asdsad,JUNHO,asdsad
3,4,COOPERBUS,1,asdsad,asdsad,asdsad,JUNHO,asdasd
4,5,INBRASP,9,asddsa,asdasd,assda,JUNHO,asdsad


#### Edita a planilha

In [372]:
planilha_anexo_modificado = planilha_anexo.drop(['ASDAD','ASDASD','ASDASD.1','ASDSAD'], axis=1)
planilha_anexo_modificado

,ID,NOME,QUANTIDADE,PERIODO
0,1,CAIO,5,JUNHO
1,2,BUSSCAR,3,JUNHO
2,3,FIBERBUS,5,JUNHO
3,4,COOPERBUS,1,JUNHO
4,5,INBRASP,9,JUNHO


#### Cria arquivo Excel

In [373]:
data_atual = datetime.now()
data_atual = data_atual.strftime('%d-%m-%Y')
arquivo_nome = f'arquivo{data_atual}.xlsx'

# Caminho para salvar o arquivo Excel
caminho_arquivo = f'C:\\GitHub\Automacao_Email_Python\\{arquivo_nome}'  # Substitua com o caminho real

# Salvar o DataFrame em um arquivo Excel
planilha_anexo_modificado.to_excel(caminho_arquivo, index=False)

# Enviar E-mail (Gmail)

In [374]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

#### Enviar E-mail com Anexo

In [377]:
destinos = input("Digite o Email que será enviado: ")

def enviar_email():
    # Configurações do e-mail
    corpo_email = """
    <p>Email enviado automaticamente usando <b>Python</b></p>
    <p>Contém <b>Anexo</b></p>
    """
    
    # Criar mensagem multipart
    msg = MIMEMultipart()
    msg['Subject'] = 'Email Automático usando Python'
    msg['From'] = usuario
    msg['To'] = destinos
    # msg['To'] = 'brunacamsilva@gmail.com'

    # Corpo do e-mail
    msg.attach(MIMEText(corpo_email, 'html'))
    
    with open(caminho_arquivo, 'rb') as anexo_arquivo:
        anexo = MIMEApplication(anexo_arquivo.read(), _subtype='xlsx')
        anexo.add_header('Content-Disposition', 'attachment', filename=arquivo_nome)
        msg.attach(anexo)

    # Enviar o e-mail
    with smtplib.SMTP('smtp.gmail.com', 587) as s:
        s.starttls()
        s.login(usuario, senha)
        s.send_message(msg)
    
    print('Email Enviado')

In [378]:
enviar_email()

Email Enviado
